## 1. 환경 설정

`(1) Env 환경변수`

In [25]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

`(2) 라이브러리`

In [26]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. 도구 호출 (Tool Calling)
- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음 

### 2-1. 랭체인 내장 도구
- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

**Tavily란?**
> LLM이 인터넷에서 '똑똑하게' 정보를 찾아와서 답변의 질을 높이거나 특정 작업을 더 잘 수행할 수 있도록 돕는 전문 검색 도구

In [27]:
from langchain_community.tools import TavilySearchResults
from langchain_tavily.tavily_search import TavilySearch

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearchResults(max_results=2)

# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력
for result in search_results:
    print(result)  
    print("-" * 100)  

{'title': '등심 스테이크에는 이 와인 드셔보세요! 소고기 부위별 레드와인 추천 ...', 'url': 'https://m.blog.naver.com/wineislikeacat/223096696241', 'content': '중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\n\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\n\n\u200b\n\n이에 맞는 레드와인으로는\n\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\n\n\u200b\n\n![Image 7](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTEw/MDAxNjgzNTQ1MDQ3NDI0.v4ZHSvlETicMdag0DaoWGFsrTMFezoiZD_41J0Htwhsg.mPiLhr9soLZMXEOb_GJatpnqVUUNFZkrcmQRn9omaewg.JPEG.wineislikeacat/clark-douglas-7WJV-SDSj6c-unsplash.jpg?type=w80_blur)\n\n안심 스테이크와 어울리는 와인\n\n> **안심 스테이크와 어울리는 와인 품종은? 산지오베제!**\n\n고기 본연의 맛을 즐기기 가장 좋은 부위로 꼽히는 안심은 [...] ![Image 6](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTQ4/MDAxNjgzNTQ0MjY4OTc2.FasX2MpcBaCDnf0qjWA_PcbLTIlySdPLi7FFnJlTboog.AQwrp1etohVQ3fiIHA0ga9lD45uGexCOZzB_1C12eGQg.JPEG.wineislikeacat/paul-einerhand-dwtt3noJG2k-unsplash.jpg?type=w800)\n\n등심 & 채끝 스테이크와 어울리는 와인\n\n> **등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!**\n\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\

In [28]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search_results_json
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. Input '
 'should be a search query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}
----------------------------------------------------------------------------------------------------

`(2) 도구(tool) 호출하기`

In [29]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3
)

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])

In [30]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)
print(type(ai_msg))

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

<class 'langchain_core.messages.ai.AIMessage'>
AIMessage(content='안녕하세요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 261, 'total_tokens': 271, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.221979037, 'prompt_time': 0.008821192, 'completion_time': 0.020139784, 'total_time': 0.028960976}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_79da0e0073', 'id': 'chatcmpl-453da75e-3ddc-4c63-b00f-5f5271df06ec', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--fbd60cc6-01ae-41b8-96f4-4696b5b107c9-0', usage_metadata={'input_tokens': 261, 'output_tokens': 10, 'total_tokens': 271, 'input_token_details': {}, 'output_token_details': {}})
----------------------------------------------------------------------------------------------------
'안녕하세요. 어떻게 도와드릴까요?'
------------------------------------------------------------------

In [31]:
from pprint import pprint

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# AIMessage의 속성 확인
pprint(vars(ai_msg))


{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"query":"스테이크와 '
                                                                 '어울력는 와인 추안"}',
                                                    'name': 'tavily_search_results_json'},
                                       'id': '7cjpbjdbv',
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run--c2be152a-d1b3-4da5-bf0a-34ddcdb13d5c-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': 'chatcmpl-d69f3ea7-7a24-47aa-be51-80069d935888',
                       'logprobs': None,
                       'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct',
                       'service_tier': None,
                       'system_fingerprint': 'fp_37da608fc1',
                       'token_usage': {'completion_time': 0.129746076,
                    

In [32]:

pprint(ai_msg)
print("#" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("*" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '7cjpbjdbv', 'function': {'arguments': '{"query":"스테이크와 어울력는 와인 추안"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 270, 'total_tokens': 334, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.9667425260000001, 'prompt_time': 0.013391176, 'completion_time': 0.129746076, 'total_time': 0.143137252}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'id': 'chatcmpl-d69f3ea7-7a24-47aa-be51-80069d935888', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c2be152a-d1b3-4da5-bf0a-34ddcdb13d5c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울력는 와인 추안'}, 'id': '7cjpbjdbv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 270, 'output_tokens': 64, 'total_tokens': 334, 'input_tok

In [33]:
tool_call = ai_msg.tool_calls[0]

print(tool_call)

{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울력는 와인 추안'}, 'id': '7cjpbjdbv', 'type': 'tool_call'}


`(3) 도구(tool) 실행하기`

#### 방법 1: 직접 도구 호출 처리한 결과로 ToolMessage 객체 생성

* 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
* 'args'를 사용하여 도구를 호출하고 결과를 얻는다.
* 도구 호출 결과를 사용하여 ToolMessage 객체를 생성한다.
* 도구 호출의 ID와 이름을 포함하여 더 구조화된 ToolMessage 메시지를 만든다.

In [34]:
tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)

pprint(tool_output)

tavily_search_results_json 호출 결과:
----------------------------------------------------------------------------------------------------
[{'content': '1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n'
             '------------------------------------------------\n'
             '\n'
             '![Image '
             '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
             '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n'
             '\n'
             '**추천 이유:**\n'
             '\n'
             '카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 '
             '이루며, 고기의 감칠맛을 더욱 살려줍니다.\n'
             '\n'
             '**어울리는 스테이크:**\n'
             '\n'
             '*   리브아이 스테이크\n'
             '*   뉴욕 스트립\n'
             '*   티본 스테이크\n'
             '\n'
             '**추천 와인:**\n'
             '\n'
             '*   나파 밸리 카베르네 소비뇽\n'
             '*   샤또 마고 (Château Margaux) [...] !

In [35]:

from langchain_core.messages import ToolMessage

# [{'args': {'query': '스테이크와 어울리는 와인 추천', 'search_depth': 'advanced'},
#   'id': 'call_j3TZ15VKG15fIxIpvqM4h4Dz',
#   'name': 'tavily_search',
#   'type': 'tool_call'}]

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

pprint(tool_message.content)

[{'content': '1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n'
             '------------------------------------------------\n'
             '\n'
             '![Image '
             '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
             '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n'
             '\n'
             '**추천 이유:**\n'
             '\n'
             '카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 '
             '이루며, 고기의 감칠맛을 더욱 살려줍니다.\n'
             '\n'
             '**어울리는 스테이크:**\n'
             '\n'
             '*   리브아이 스테이크\n'
             '*   뉴욕 스트립\n'
             '*   티본 스테이크\n'
             '\n'
             '**추천 와인:**\n'
             '\n'
             '*   나파 밸리 카베르네 소비뇽\n'
             '*   샤또 마고 (Château Margaux) [...] ![Image '
             '10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image '
             '11](h

#### 방법 2: 도구 직접 호출하여 바로 ToolMessage 객체 생성

* 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
* 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

In [36]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

In [37]:
tool_call = ai_msg.tool_calls[0]

print(tool_call)

{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울력는 와인 추안'}, 'id': '3gdxrv2qj', 'type': 'tool_call'}


In [38]:
# tool_call  {'name': 'tavily_search_results_json', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_LrHyxTadqHDjW7J6LOWEaoSi', 'type': 'tool_call'}
tool_message = web_search.invoke(tool_call)
print(type(tool_message))

# 특정 속성들만 확인
print("\n=== 주요 속성들 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")


<class 'langchain_core.messages.tool.ToolMessage'>

=== 주요 속성들 ===
content: [{"title": "스테이크와 찰떡궁합! 스테이크에 어울리는 와인 5가지 - 술의 모든 것", "url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n------------------------------------------------\n\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n\n**추천 이유:**\n\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\n\n**어울리는 스테이크:**\n\n*   리브아이 스테이크\n*   뉴욕 스트립\n*   티본 스테이크\n\n**추천 와인:**\n\n*   나파 밸리 카베르네 소비뇽\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)!

In [39]:
pprint(tool_message.content)

('[{"title": "스테이크와 찰떡궁합! 스테이크에 어울리는 와인 5가지 - 술의 모든 것", "url": '
 '"https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", '
 '"content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 '
 '대표주자\\n------------------------------------------------\\n\\n![Image '
 '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
 '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 '
 '이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, '
 '고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   '
 '티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château '
 'Margaux) [...] ![Image '
 '10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image '
 '11](h

In [40]:
pprint(tool_message.tool_call_id)

'3gdxrv2qj'


In [41]:
pprint(tool_message.name)

'tavily_search_results_json'


In [42]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': '스테이크와 어울력는 와인 추안'},
  'id': '3gdxrv2qj',
  'type': 'tool_call'}]

In [43]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='[{"title": "스테이크와 찰떡궁합! 스테이크에 어울리는 와인 5가지 - 술의 모든 것", "url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\\n------------------------------------------------\\n\\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net

`(4) ToolMessage를 LLM에 전달하여 답변을 생성하기`

* 파라미터 이름이 config여야 하는 이유: @chain 데코레이터가 config라는 이름만 특별히 인식하도록 설계되었습니다.
* config 생략 가능한 이유: @chain이 자동으로 기본 RunnableConfig()를 생성해서 전달하기 때문입니다.

In [46]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
# llm = ChatOpenAI(model="gpt-4o-mini")

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
tavily_tool = TavilySearchResults(max_results=2)

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_tool])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
# @chain 은 일반 함수를 Runnable 객체로 만들어 주는 데코레이터
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = tavily_tool.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "스테이크와 어울리는 와인을 추천해주세요."
response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: gsk_sfJa********************************************8nq0. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### 2-2. 사용자 정의 도구
- @tool decorator를 통해 사용자 정의 도구를 정의할 수 있음

`(1) 도구(tool) 정의하기`

In [ ]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

### 2-3. Runnable 객체를 도구(tool) 변환
- 문자열이나 dict 입력을 받는 Runnable을 도구로 변환
- as_tool 메서드를 사용

- wikipedia 설치
```
    poetry add wikipedia
```
- [WikipediaLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html) 사용


In [ ]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)

In [ ]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_search
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a specified number of documents. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'k': {'default': 2,
                      'description': 'The number of documents to return '
                                     '(default is 2)',
                      'title': 'K',
                      'type': 'integer'},
                'q

In [ ]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

page_content='투움바 파스타(영어: Toowoomba pasta)는 미국의 파스타 요리이며,
이름은 오스트레일리아의 도시인 터움바에서 유래되었다.


== 역사 ==
미국 아웃백 스테이크하우스의 메뉴로 만들어졌으나, 미국에서는 단종되었으며 한국과 브라질 등지에서는 메뉴로 남아 있다. 특히 한국에서 크게 인기를 얻었으며, 한국 매체나 유튜버에 의해 세계에 알려지면서 한국 요리로 소개되거나 외국의 한국 음식점에서 판매되기도 한다.


== 만들기 ==
페투치네 등 두꺼운 파스타에 고춧가루를 넣어 매콤한 크림 소스를 추가한 음식이다.


== 활용 ==
투움바 소스를 활용한 음식으로 투움바 떡볶이, 투움바 라면, 투움바 치킨 등이 있다.


== 같이 보기 ==
아라비아타


== 각주 ==' metadata={'title': '투움바 파스타', 'summary': '투움바 파스타(영어: Toowoomba pasta)는 미국의 파스타 요리이며,\n이름은 오스트레일리아의 도시인 터움바에서 유래되었다.', 'source': 'https://ko.wikipedia.org/wiki/%ED%88%AC%EC%9B%80%EB%B0%94_%ED%8C%8C%EC%8A%A4%ED%83%80'}
----------------------------------------------------------------------------------------------------
page_content='피치(이탈리아어: pici, 단수: picio 피초[*])는 이탈리아의 파스타이다. 손으로 말아서 만드는 굵은 파스타의 일종으로 스파게티면이 좀 더 굵어진 것으로 보면 된다. 토스카나주의 시에나 현에서 유래했으며 몬탈치노 지역에서는 pinci라고 부른다.
반죽은 보통 밀가루나 물로만 만든다. 달걀을 첨가하는 것은 선택적이며 가정에 따라 다르다.
밀가루 반죽을 두껍고 평평하게 밀어서 편 다음 기다란 조각으로 잘라낸다. 잘라낸 조각을 두 손바닥 사이에서 말기도 하고 테이블 위에 놓고 

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ltcFrZ7FfdvR33bjP998f9pL', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_IPqif944DodK4SvRmmcMtJg6', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 176, 'total_tokens': 234, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiwRSoRRebiugoTD3ZVe50N9rQqFI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--95f2c707-66dd-42f7-8a19-9be64f3619f1-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'i

`(2) LCEL 체인`
- 위키피디아 문서를 검색하고 내용을 요약하는 체인

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

('투움바 파스타는 미국의 파스타 요리로, 오스트레일리아의 투움바에서 유래되었다. 미국 아웃백 스테이크하우스에서 처음 만들어졌으나 현재는 '
 '한국과 브라질에서 인기를 끌고 있다. 이 요리는 두꺼운 파스타에 매콤한 크림 소스를 추가하여 만든다. 이와 관련된 다양한 요리도 '
 '존재한다.\n'
 '\n'
 '이탈리아 요리는 기원전 4세기부터 발전해온 남유럽의 전통 요리로, 지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 '
 '해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, CNN에 의해 세계 최고의 요리로 선정된 바 있다. 이 요리는 '
 '역사적으로 다양한 문화의 영향을 받아 발전해왔으며, 고대 그리스와 로마의 요리법이 포함되어 있다.')


In [ ]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [ ]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Pqm9U9nGCURVGiXidbW5qwdB', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_LjwvfwCR5WieqAR85VA3t1Nw', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 152, 'total_tokens': 210, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiGzNI0bcEgWNBCbZjxbfl7mgw69C', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--40eb99e9-fdef-47d7-96f7-1af205a24591-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, '

In [ ]:
ai_msg.tool_calls[1]

{'name': 'wiki_search',
 'args': {'query': '파스타의 유래'},
 'id': 'call_IPqif944DodK4SvRmmcMtJg6',
 'type': 'tool_call'}

In [ ]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])

print(tool_message)
print("-" * 100)
pprint(tool_message.content)

content='투움바 파스타는 미국의 파스타 요리로, 오스트레일리아의 투움바에서 유래되었다. 미국 아웃백 스테이크하우스에서 처음 만들어졌으나 현재는 단종되었고, 한국과 브라질에서 여전히 인기 있는 메뉴로 남아 있다. 이 요리는 두꺼운 파스타에 매콤한 크림 소스를 추가하여 만든다. 오르조는 이탈리아의 파스타로, 보리 모양의 작은 형태를 가지고 있으며, 주로 수프와 함께 먹는다. 오르조는 다양한 이름으로 불리며, 터키에서는 아르파 셰흐리예로 알려져 있다.' name='wiki_summary' tool_call_id='call_IPqif944DodK4SvRmmcMtJg6'
----------------------------------------------------------------------------------------------------
('투움바 파스타는 미국의 파스타 요리로, 오스트레일리아의 투움바에서 유래되었다. 미국 아웃백 스테이크하우스에서 처음 만들어졌으나 현재는 '
 '단종되었고, 한국과 브라질에서 여전히 인기 있는 메뉴로 남아 있다. 이 요리는 두꺼운 파스타에 매콤한 크림 소스를 추가하여 만든다. '
 '오르조는 이탈리아의 파스타로, 보리 모양의 작은 형태를 가지고 있으며, 주로 수프와 함께 먹는다. 오르조는 다양한 이름으로 불리며, '
 '터키에서는 아르파 셰흐리예로 알려져 있다.')


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_oz3XdrweY14cUDCTurHbIfcO', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BiwRw6qLdKr0Ggvjij7ge8eCHXN4S', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--72cb6e50-f3ef-4489-a281-a98947dbc779-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_oz3XdrweY14cUDCTurHbIfcO', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139, 'input_token_details': {

### 2-4. 벡터저장소 검색기
- @tool decorator 사용

`(1) 문서 로드 및 인덱싱`

In [ ]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [ ]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...

메뉴 번호: 2
메뉴 이름: 트러플 리조또
내용:
2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


### Embedding을 하기 위해 Ollama Qwen2.5 설치
```
ollama run qwen2.5:1.5b

ollama ls
```

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="qwen2.5:1.5b") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("./db/menu_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 2개
메뉴 번호: 3
메뉴 이름: 연어 타르타르

메뉴 번호: 2
메뉴 이름: 트러플 리조또



- 와인 메뉴에 대해서도 같은 작업을 처리

In [ ]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("../data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# FAISS 인덱스 생성
wine_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

wine_db.save_local("./db/wine_db")

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...
검색 결과: 2개
메뉴 번호: 10
메뉴 이름: 그랜지 2016

메뉴 번호: 6
메뉴 이름: 바롤로 몬프리바토 2017



`(2) 도구(tool) 정의하기`

In [ ]:
from langchain_core.tools import tool
from typing import List

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "./db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_menu_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_menu_func))
print("-"*100)

print("name: ")
print(db_search_menu_func.name)
print("-"*100)

print("description: ")
pprint(db_search_menu_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_menu_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_menu_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant menu '
                'information from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_menu_func',
 'type': 'object'}


In [ ]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# wine db 벡터 저장소 로드
wine_db = FAISS.load_local(
    "./db/wine_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_wine_func(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_wine_func))
print("-"*100)

print("name: ")
print(db_search_wine_func.name)
print("-"*100)

print("description: ")
pprint(db_search_wine_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_wine_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_wine_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant wine information from the '
 'encrypted database.\n'
 'Use this tool only for wine-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant wine '
                'information from the encrypted database.\n'
                'Use this tool only for wine-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_wine_func',
 'type': 'object'}


In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[db_search_menu_func, db_search_wine_func])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 추천 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XnvGZbno1nzYyT4exShN1aXM', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'db_search_menu_func'}, 'type': 'function'}, {'id': 'call_LLJ7LZdIJZt4wI5BWwpcVeuw', 'function': {'arguments': '{"query": "와인 추천"}', 'name': 'db_search_wine_func'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 133, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Biwi5lvUTGlrtAFdUx8isKQdWGnur', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e56a43fd-05f4-4f2e-9b1c-4770f4f41043-0', tool_calls=[{'name': 'db_search_menu_func', 'args': {'query': '시그니처 스테이크'}, 

`(3) 여러 개의 도구(tool) 호출하기`

In [ ]:
# @tool : tavily_search_func, db_search_menu_func, db_search_wine_func
# chain.as_tool() : wiki_summary
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
for tool in tools:
    print(tool.name)

tavily_search_func
wiki_summary
db_search_menu_func
db_search_wine_func


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from pprint import pprint

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    for tool in tool_msgs:
        print(tool.name)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 해주세요.")

# 응답 출력 
print(response.content)

db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_MnPGlaf221crQAKDgAXQxIUz', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
db_search_wine_func: 
{'name': 'db_search_wine_func', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_7kTJVaqmHQqCh4CFLflbGk5F', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='e98025c4-f4f8-4aa9-8426-5b4e48bfd340', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='1ae85f26-c8c3-4c54-af2c-f5f840ebb890', metadata={'source': '../

In [ ]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 파스타의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '파스타'}, 'id': 'call_jvIy2VAyToYyBUFSYv4DOt0d', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타의 역사'}, 'id': 'call_hnKMMDQZ24SWn6oi3Q74HmE7', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='b6865964-9504-4552-88a2-d9385c240b46', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 8, 'menu_name': '안심 스테이크 샐러드'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격: ₩26,000\\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.'), Document(id='e3656bb9-71b4-412e-a75f-7134d0248eaf', metadata={'source': './data/restaurant_menu.txt', 'menu_number': 6

## 3. Few-shot 프롬프팅 
- 각 도구의 용도를 구분하여 few-shot 예제로 제시

### 3-1. Few-shot 도구 호출

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_menu_func", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),    
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. \
                주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_wine_func", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. \
                2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다.\
               주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, \
              주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. \
              특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오,\
               그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""


In [ ]:

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인도 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': 'call_77KuFqh1SWzPmKZwJPk2tqg4', 'type': 'tool_call'}
{'name': 'db_search_wine_func', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_zme3gvxATMMsuIotwDr5sPUv', 'type': 'tool_call'}


In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_oIQKzQfkPtu8QlS3oNSyXhU4', 'type': 'tool_call'}
{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_UgqWlO41rHYJrS1uEwsNWBhk', 'type': 'tool_call'}


### 3-2. 답변 생성 체인 

In [ ]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
# llm = ChatOpenAI(model="gpt-4o-mini")

# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해 주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': 'call_vOMBMyf4kpzNgwUPBjJNnn8b', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
db_search_wine_func: 
{'name': 'db_search_wine_func', 'args': {'query': '스테이크'}, 'id': 'call_nyQ1wH0DUzT9LlbuAUJiLd2R', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='2b0ebfad-94fe-440a-b893-2dd2a723c926', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 8, 'menu_name': '안심 스테이크 샐러드'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격: ₩26,000\\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.'), Document(id='e98025c4-f4f8-4aa9-8426-5b4e48bfd340', metadata={'source': '../data/restaurant_menu.txt', 'm

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_2FqpHW8uaQr1i17xCUGOLMzh', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tavily_search_func: 
{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_RrfuLekfao6tSqS9kcU7zwqR', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------


C:\Users\sogno\AppData\Local\Temp\ipykernel_20292\3430203717.py:9: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=2)


tool_msgs: 
 [ToolMessage(content='투움바 파스타는 미국의 아웃백 스테이크하우스에서 유래된 요리로, 오스트레일리아의 투움바에서 이름을 따왔으며, 매콤한 크림 소스를 사용한 두꺼운 파스타이다. 한국과 브라질에서 인기를 끌고 있으며, 다양한 변형 요리도 존재한다. 피치는 이탈리아의 전통 파스타로, 손으로 만들어지는 굵은 형태로, 주로 밀가루와 물로 반죽하여 만든다. 주로 토스카나 지역에서 유래하며, 다양한 요리에 활용된다.', name='wiki_summary', tool_call_id='call_2FqpHW8uaQr1i17xCUGOLMzh'), ToolMessage(content='<Document href="https://todflrh06.tistory.com/3"/>\n서울 강남 파스타 맛집 추천 BEST 5 · 1. 라비타 (La Vita) · 2. 볼라레 (VolaRe) · 3. 더 테이블 (The Table) · 4. 오스테리아 밀라노 (Osteria Milano).\n</Document>\n---\n<Document href="https://blog.naver.com/pronl5v2/223447446768"/>\n신논현역 6번 출구에서 317m만 걸어 오면 바로 보이는 강남 파스타 맛집 바비레드 BOBIRED ! \u200b. 바비레드 강남본점. 서울특별시 강남구 봉은사로6길 39\n</Document>', name='tavily_search_func', tool_call_id='call_RrfuLekfao6tSqS9kcU7zwqR')]
----------------------------------------------------------------------------------------------------
('### 파스타의 유래\n'
 '파스타는 이탈리아의 밀가루 반죽 식품으로, 면의 종류에 따라 다양한 형태와 요리법이 있습니다. 일반적으로 이탈리아에서 유래했지만, '
 '북아프리카에서 유래된 이탈리아의 전통적인 

## 4. LangChain Agent 사용
- 유의사항: 프롬프트에 "agent_scratchpad",  "input" 변수를 포함

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# AgentExecutor 실행

query = "안심 스테이크의 가격과 특징은 무엇인가요? 그리고 와인도 추천해주세요."
agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `db_search_menu_func` with `{'query': '안심 스테이크'}`


[Document(id='e98025c4-f4f8-4aa9-8426-5b4e48bfd340', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='2b0ebfad-94fe-440a-b893-2dd2a723c926', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 8, 'menu_name': '안심 스테이크 샐러드'}, page_content='8. 안심 스테이크 샐러드\n   • 가격: ₩26,000\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.')]
Invoking: `db_search_wine_func` with `{'query': '안심 스테이크'}`


[Document(id='9337b6c5-4bb5-4760-a3c0-d832976c8cf4', metadata={'source'

In [ ]:
pprint(agent_response)

{'input': '시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.',
 'output': '### 시그니처 스테이크 정보\n'
           '\n'
           '- **가격**: ₩35,000\n'
           '- **주재료**: 소고기, 로즈마리, 허브 믹스\n'
           '- **특징**: 신선한 질 좋은 소고기를 사용하여 은은한 로즈마리 향과 허브 믹스를 곁들여 조리합니다. 반숙 달걀과 '
           '함께 제공되어 특별한 맛을 더합니다. 특히, 스테이크의 식감과 맛을 최대로 느낄 수 있도록 조리되어 있습니다.\n'
           '\n'
           '### 스테이크 와인 추천\n'
           '\n'
           '1. **포르트 와인 1er Cru 2018**\n'
           '   - **가격**: ₩320,000\n'
           '   - **주요 특징**: 진한 향미와 다양한 과일의 맛을 가지고 있어 스테이크와의 조화가 뛰어납니다.\n'
           '  \n'
           '2. **그릇지 2016**\n'
           '   - **가격**: ₩950,000\n'
           '   - **주요 특징**: 복합적인 맛과 풍부한 향이 특징이며, 스테이크의 풍미를 더욱 돋보이게 만들어 줍니다.\n'
           '\n'
           '스테이크와 잘 어울리는 와인 중 포르트 와인은 그 자체로도 풍미가 깊어, 고기의 짭짤함과 좋은 균형을 이룹니다. 반면 '
           '그릇지는 더욱 고급스러운 경험을 제공하여 특별한 날에 잘 어울립니다. 어떤 선택을 하시든 멋진 식사를 즐기실 수 '
           '있습니다!'}


## 5. Gradio 활용

In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

In [ ]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `db_search_wine_func` with `{'query': '트러플 리조또'}`


[Document(id='389dc8a9-475e-4d1a-9409-a033904537ab', metadata={'source': './data/restaurant_wine.txt', 'menu_number': 10, 'menu_name': '그랜지 2016'}, page_content='10. 그랜지 2016\n    • 가격: ₩950,000\n    • 주요 품종: 시라\n    • 설명: 호주의 대표적인 아이콘 와인입니다. 블랙베리, 자두, 블랙 올리브의 강렬한 과실향과 함께 유칼립투스, 초콜릿, 가죽의 복잡한 향이 어우러집니다. 풀바디이며 강렬한 타닌과 산도가 특징적입니다. 놀라운 집중도와 깊이, 긴 여운을 자랑하며, 수십 년의 숙성 잠재력을 가집니다.'), Document(id='601e6f15-4d58-427b-a070-a7161782c2f9', metadata={'source': './data/restaurant_wine.txt', 'menu_number': 7, 'menu_name': '풀리니 몽라쉐 1er Cru 2018'}, page_content='7. 풀리니 몽라쉐 1er Cru 2018\n   • 가격: ₩320,000\n   • 주요 품종: 샤르도네\n   • 설명: 부르고뉴 최고의 화이트 와인 중 하나로 꼽힙니다. 레몬, 사과, 배의 과실향과 함께 헤이즐넛, 버터, 바닐라의 풍부한 향이 어우러집니다. 미네랄리티가 돋보이며, 크리미한 텍스처와 긴 여운이 특징입니다. 해산물, 닭고기, 크림 소스 파스타와 좋은 페어링을 이룹니다.')]트러플 리조또와 잘 어울리는 와인으로는 **풀리니 몽라쉐 1er Cru 2018**를 추천드립니다. 가격은 ₩320,000입니다.

이 와인은 부르고뉴 최고의 화이트 와인 중 하나로, 레몬, 사과, 배의 과일 향이 돋보이며, 

In [ ]:
# 데모 종료
demo.close()

Closing server running on port: 7860
